In [2]:
from sklearn.datasets import fetch_20newsgroups

In [5]:
data = fetch_20newsgroups(subset='train', 
                          categories=['comp.sys.mac.hardware', 'soc.religion.christian', 'rec.sport.hockey'])

In [12]:
dir(data)

['DESCR', 'data', 'filenames', 'target', 'target_names']

In [25]:
simple_data = data.data

In [26]:
print(simple_data[1])

From: drlovemd@jhuvms.hcf.jhu.edu (Steve Liu)
Subject: What is the current Rom Version shipping in Syquest drives?
Organization: The Johns Hopkins University - HCF
Lines: 4
Distribution: world
NNTP-Posting-Host: jhuvms.hcf.jhu.edu
News-Software: VAX/VMS VNEWS 1.41    

The title says it all.  I need to know the 44, 88, and 88c rom versions.

Steve :-)




In [27]:
len(simple_data)

1777

In [23]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [29]:
vectorizer = CountVectorizer(max_df=500, min_df=10) #максимальная и минимальная встречаемость слова в текстах
matrix = vectorizer.fit_transform(simple_data)

In [47]:
matrix

<1777x3767 sparse matrix of type '<class 'numpy.int64'>'
	with 157682 stored elements in Compressed Sparse Row format>

# Agglomerative Clustering

In [41]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import accuracy_score
import pandas as pd

In [33]:
model = AgglomerativeClustering(n_clusters=3, affinity='cosine', linkage='complete')
predictions = model.fit_predict(matrix.toarray())

In [42]:
pd.value_counts(predictions)

0    1705
1      44
2      28
dtype: int64

In [43]:
pd.value_counts(data.target)

1    600
2    599
0    578
dtype: int64

In [45]:
vectorizer_tfidf = TfidfVectorizer(max_df=500, min_df=10)
matrix_tfidf = vectorizer_tfidf.fit_transform(simple_data)

In [48]:
predictions_tfidf = model.fit_predict(matrix_tfidf.toarray())

In [49]:
accuracy_score(data.target, predictions_tfidf)

0.325267304445695

In [51]:
pd.value_counts(predictions_tfidf)

0    1717
2      39
1      21
dtype: int64

In [53]:
print(matrix[0])

  (0, 1246)	3
  (0, 239)	1
  (0, 1659)	1
  (0, 2082)	1
  (0, 1846)	1
  (0, 2991)	4
  (0, 810)	1
  (0, 2056)	1
  (0, 319)	1
  (0, 625)	1
  (0, 3553)	1
  (0, 1121)	1
  (0, 838)	1
  (0, 69)	1
  (0, 2413)	1
  (0, 3555)	1
  (0, 3600)	1
  (0, 2273)	1
  (0, 1905)	1
  (0, 3722)	1
  (0, 926)	1
  (0, 1991)	1
  (0, 2048)	1
  (0, 354)	1
  (0, 900)	1
  :	:
  (0, 2868)	1
  (0, 855)	3
  (0, 301)	1
  (0, 2422)	1
  (0, 3285)	1
  (0, 1831)	1
  (0, 620)	1
  (0, 2325)	1
  (0, 3322)	1
  (0, 294)	1
  (0, 619)	2
  (0, 1961)	2
  (0, 551)	1
  (0, 3305)	1
  (0, 1366)	1
  (0, 3262)	1
  (0, 1229)	1
  (0, 3035)	1
  (0, 2108)	1
  (0, 2820)	1
  (0, 2358)	1
  (0, 1136)	1
  (0, 2243)	1
  (0, 880)	1
  (0, 877)	1


In [60]:
vectorizer.get_feature_names()[855]

'computer'

In [58]:
print(simple_data[0])

From: erik@cheshire.oxy.edu (Erik Adams)
Subject: HELP!!  My Macintosh "luggable" has lines on its screen!
Organization: Occidental College, Los Angeles, CA 90041 USA.
Distribution: comp
Lines: 20

Okay, I don't use it very much, but I would like for it to keep working
correctly, at least as long as Apple continues to make System software
that will run on it, if slowly :-)

Here is the problem:  When the screen is tilted too far back, vertical
lines appear on the screen.  They are every 10 pixels or so, and seem
to be affected somewhat by opening windows and pulling down menus.
It looks to a semi-technical person like there is a loose connection
between the screen and the rest of the computer.

I am open to suggestions that do not involve buying a new computer,
or taking this one to the shop.  I would also like to not have
to buy one of Larry Pina's books.  I like Larry, but I'm not sure
I feel strongly enough about the computer to buy a service manual
for it.

On a related note:  what

In [ ]:
from sklearn.cluster import KMeans

In [64]:
kmeans = KMeans(n_clusters=3, random_state=42)
predictions = kmeans.fit_predict(matrix_tfidf.toarray())

In [65]:
accuracy_score(data.target, predictions)

0.9527293190770962

In [70]:
kmeans.labels_

array([0, 0, 1, ..., 0, 1, 2])

In [71]:
data_hard = fetch_20newsgroups(categories=['comp.sys.mac.hardware', 'comp.os.ms-windows.misc', 'comp.graphics'])

In [72]:
data_hard_X = data_hard.data

In [79]:
matrix_hard = vectorizer_tfidf.fit_transform(data_hard_X)

In [81]:
predictions_hard = kmeans.fit_predict(matrix_hard.toarray())

In [92]:
predictions_hard

array([2, 1, 0, ..., 2, 0, 2])

In [93]:
data_hard.target

array([2, 1, 1, ..., 2, 0, 2], dtype=int64)

In [94]:
accuracy_score(predictions_hard, data_hard.target)

0.7769537934968626

# SVM + KMeans

In [95]:
from sklearn.decomposition import TruncatedSVD

In [129]:
svd = TruncatedSVD(n_components=250, random_state=123)
features = svd.fit_transform(matrix_hard.toarray())

In [130]:
preds_svm = kmeans.fit_predict(features)

In [103]:
import itertools

In [131]:
for a, b, c in itertools.permutations([0, 1, 2]): 
    mapping = {a: 0, b: 1, c: 2}
    preds_replaced = [mapping[val] for val in preds_svm]
    print((a, b, c), accuracy_score(data_hard.target, preds_replaced))

(0, 1, 2) 0.11466058185966914
(0, 2, 1) 0.4249857387335995
(1, 0, 2) 0.26640045636052484
(1, 2, 0) 0.1677124928693668
(2, 0, 1) 0.7176269252709641
(2, 1, 0) 0.30861380490587564
